In [1]:
!pip install -U transformers


In [3]:
# Making some essential imports as Usual
import os
import sys
import numpy as np
import re
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import pipeline

In [4]:
captioner = pipeline("image-to-text",model="Salesforce/blip-image-captioning-large", device=0)

In [57]:
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(folder_path)

OSError: [Errno 16] Device or resource busy: '/kaggle/working'

In [5]:
!mkdir /kaggle/working/source
!mkdir /kaggle/working/target

In [6]:
!touch /kaggle/working/json_output.txt

In [8]:
# defining function for dataLoading function
framObjTrain = {'img' : [],
           'mask' : [],
            'caption': []
          }



def ProcessData( frameObj = None, imgPath = None, maskPath = None, shape = 256):
    json_write = ""
    imgNames = os.listdir(imgPath)
    maskNames = []
    
    ## generating mask names
    for mem in imgNames:
        mem = mem.split('_')[0]
        if mem not in maskNames:
            maskNames.append(mem)
    
    imgAddr = imgPath + '/'
    maskAddr = maskPath + '/'
    
    for i in range (len(imgNames)):
        try:
            img = plt.imread(imgAddr + maskNames[i] + '_sat.jpg') 
            mask = plt.imread(maskAddr + maskNames[i] + '_mask.png')
            
        except:
            continue
            
        kernel = np.ones((3, 3), np.uint8)
        img = cv2.resize(img, (shape, shape))
        mask = cv2.dilate(cv2.resize(mask, (shape, shape)), kernel, iterations=1)
        frameObj['img'].append(img)
        frameObj['mask'].append(mask) # this is because its a binary mask and img is present in channel 0
        
        # we've generated the captions once now, no need to do it again 
        cap = captioner(imgAddr + maskNames[i] + '_sat.jpg')[0]['generated_text']
        text = "{\"source\": \"source/" + maskNames[i] + ".jpg\", \"target\": \"target/" + maskNames[i] + ".png\", \"prompt\": \"" + cap + "\"}"
        frameObj['caption'].append(text)

        cv2.imwrite("/kaggle/working/source/" + maskNames[i] + ".jpg", mask)
        cv2.imwrite("/kaggle/working/target/" + maskNames[i] + ".jpg", img)
        json_write = json_write + text + "\n"
        
    return frameObj

In [9]:
framObjTrain = ProcessData( framObjTrain, imgPath = '/kaggle/input/deepglobe-road-extraction-dataset/train', 
                        maskPath = '/kaggle/input/deepglobe-road-extraction-dataset/train'
                         , shape = 256)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [15]:
text = ""
for val in framObjTrain['caption']:
    text = text + val + "\n"

In [17]:
with open('/kaggle/working/json_output.txt', 'w') as f:
    f.write(text)

In [25]:
!mkdir /kaggle/working/source_dilated
!mkdir /kaggle/working/target_dilated

In [35]:
testLoad = {'img' : [],
           'mask' : [],
        }


def LoadDilatedData( frameObj = None, imgPath = None, maskPath = None, shape = 256):
    json_write = ""
    imgNames = os.listdir(imgPath)
    maskNames = []
    
    ## generating mask names
    for mem in imgNames:
        mem = mem.split('_')[0]
        if mem not in maskNames:
            maskNames.append(mem)
    
    imgAddr = imgPath + '/'
    maskAddr = maskPath + '/'
    
    for i in range (min(len(imgNames), 1)):
        try:
            img = cv2.imread(imgAddr + maskNames[i] + '_sat.jpg') 
            mask = cv2.imread(maskAddr + maskNames[i] + '_mask.png', cv2.IMREAD_GRAYSCALE)
            
        except:
            print("what")
            continue
            
        kernel = np.ones((3, 3), np.uint8)
        img = cv2.resize(img, (shape, shape))
        mask = cv2.dilate(cv2.resize(mask, (shape, shape)), kernel, iterations=1)
        frameObj['img'].append(img)
        frameObj['mask'].append(mask) # this is because its a binary mask and img is present in channel 0

        
        
        cv2.imwrite("/kaggle/working/source_dilated/" + maskNames[i] + ".jpg", mask)
        cv2.imwrite("/kaggle/working/target_dilated/" + maskNames[i] + ".jpg", img)
        json_write = json_write + text + "\n"
        
    return frameObj

In [36]:
testLoad = LoadDilatedData(testLoad, imgPath = '/kaggle/input/deepglobe-road-extraction-dataset/train', 
                        maskPath = '/kaggle/input/deepglobe-road-extraction-dataset/train'
                         , shape = 256)